In [ ]:
from IPython.core.pylabtools import figsize
import pandas as pd
import numpy as np
import sys
import os
import enum
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['figure.dpi'] = 100


In [ ]:
class EjectionType(enum.Enum):
    iqr = 0
    hampel = 1


def clean_table(df: pd.DataFrame,
                path: str,
                nan_crit_val: float = 0.25,
                iqr_coef: int = 1,
                # https://leftjoin.ru/all/outliers-detection-in-python/
                ejection_type: EjectionType = EjectionType.iqr,
                save_csv: bool = False,
                do_output: bool = False):
    df = df.copy()
    log = []
    print('START data cleaning')

    for c in df.columns:
        if not pd.api.types.is_numeric_dtype(df[c]):
            df[c] = df[c].astype(str)

    # Удаление дубликатов
    df.drop_duplicates()

    # Обработка выбросов
    for c in df.columns:
        if pd.api.types.is_numeric_dtype(df[c]):
            median = df[c].median()

            if (ejection_type == EjectionType.iqr):
                Q1 = df[c].quantile(0.25)
                Q3 = df[c].quantile(0.75)
                max_deviation = (Q3 - Q1)*iqr_coef
            elif (ejection_type == EjectionType.hampel):
                difference = np.abs(median-df[c])
                max_deviation = 3 * difference.median()

            maxv = get_correct_max(df[c], max_deviation, median)
            minv = get_correct_min(df[c], max_deviation, median)

            for i in range(df[c].count()):
                v = df[c].loc[i]
                if v > median + max_deviation:
                    log.append(
                        f'in col {c} in row {i} top ejection = {v} replaced by {maxv}; md={max_deviation}, median={median}')
                    df.at[i, c] = maxv
                elif v < median - max_deviation:
                    log.append(
                        f'in col {c} in row {i} bottom ejection = {v} replaced by {minv}; md={max_deviation}, median={median}')
                    df.at[i, c] = minv

    # Обработка пропущенных значений
    for c in df.columns:
        if df[c].isna().sum() > df[c].count() * nan_crit_val:
            log.append(f'Drop column "{c}" beacuse too much nans')
            df = df.drop([c], axis=1)
        elif pd.api.types.is_numeric_dtype(df[c]):
            df[c].fillna(value=df[c].mean(), inplace=True)
        else:
            df[c].fillna(value=df[c].mode(), inplace=True)

    print('END data cleaning')
    if save_csv:
        df.to_csv(os.path.join(path, r'res.csv'))

    if do_output:
        with open(f"{path}\\temp\\log.txt", 'w') as fp:
            fp.write('\n'.join(log))
    return df


def get_correct_max(col, max_deviation, median):
    maxv = sys.float_info.min
    for v in col:
        if v > maxv:
            dif = abs(median - v)
            if dif <= max_deviation:
                maxv = v
    return maxv


def get_correct_min(col, max_deviation, median):
    minv = sys.float_info.max
    for v in col:
        if v < minv:
            dif = abs(median - v)
            if dif <= max_deviation:
                minv = v
    return minv


def make_graphs(orig: pd.DataFrame, clear: pd.DataFrame):
    cols = clear.columns

    figsize(6, 4)

    for c in cols:
        if not pd.api.types.is_numeric_dtype(orig[c]):
            continue
        print(c)
        # plt.hist([orig[c], clear[c]])
        fig, ax = plt.subplots()
        a_heights, a_bins = np.histogram(orig[c].dropna().values)
        b_heights, b_bins = np.histogram(clear[c], bins=a_bins)
        width = (a_bins[1] - a_bins[0])/3
        ax.bar(a_bins[:-1], a_heights, width=width, facecolor='cornflowerblue')
        ax.bar(b_bins[:-1]+width, b_heights, width=width, facecolor='seagreen')
        fig.suptitle(c)


In [ ]:
dict = {'First Score': [100, 90, np.nan, 95, 1, 56],
        'Second Score': [30, np.nan, 45, 56, -11111, 56],
        'Third Score': [52, 1240, 80, 98, 1, 56],
        'Fourth Score': [np.nan, np.nan, np.nan, 65, 1, 56],
        'STRS': ['52', '40', '80', None, 'aaa', 56], }

df = pd.DataFrame(dict)
directory = os.path.dirname(os.getcwd())
df = pd.read_csv(directory + '\\datasets\\avocado1.csv')
print(directory)
print(df)

df1 = clean_table(df,
                  directory,
                  do_output=True,
                  ejection_type=EjectionType.hampel)
print(df1)

make_graphs(df, df1)
